In [ ]:
#Importy i Przygotowanie Danych
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from keras import layers, models

# Ustawienie seed dla powtarzalności
np.random.seed(42)
tf.random.set_seed(42)

print(f"Wersja TensorFlow: {tf.__version__}")



In [ ]:
# --- Ładowanie danych (Iris)
data = load_iris()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

X = df.drop(columns=['target']).values
y = df['target'].values

# Standaryzacja
# Skalowanie jest kluczowe, aby cechy miały średnią 0 i odchylenie standardowe 1.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Zakres danych po skalowaniu jest potrzebny do poprawnego "klippowania"
# (przycinania) perturbacji adwersarialnych, aby zachować sensowne wartości.
min_val, max_val = X_scaled.min(), X_scaled.max()
print(f"Zakres danych po skalowaniu: [{min_val:.3f}, {max_val:.3f}]")

# --- Podział na zbiory treningowy i testowy ---
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Kształt zbioru treningowego: {X_train.shape}")

In [5]:
#Prosta sieć MLP (Multilayer Perceptron)
def create_model(input_dim):
    """Tworzy model MLP z dwiema warstwami ukrytymi."""
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),
        # Warstwa wyjściowa: 3 klasy, aktywacja softmax
        layers.Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy', # Używamy dla etykiet skalarnych (0, 1, 2)
                  metrics=['accuracy'])
    return model

model = create_model(X_train.shape[1])

#Krótki trening (Demo)
print("Rozpoczęcie treningu modelu bazowego (Model 'Clean'):")
# Używamy verbose=0, aby wyciszyć szczegóły epok w notebooku
model.fit(X_train, y_train, epochs=15, batch_size=16,
          validation_data=(X_test, y_test), verbose=0)

# Ocena modelu bazowego
res_clean_base = model.evaluate(X_test, y_test, verbose=0)
print(f"\n Dokładność modelu bazowego (Clean Test): {res_clean_base[1]:.4f}")

Rozpoczęcie treningu modelu bazowego (Model 'Clean'):

 Dokładność modelu bazowego (Clean Test): 0.8667


In [8]:
#Funkcja generująca adwersarialny przykład (FGSM)
def create_adversarial_example(model, input_data, input_label, epsilon, min_val, max_val):
    """
    Generuje adwersarialny przykład (perturbed input) metodą FGSM.

    input_data: 1D numpy array (skalowane cechy)
    input_label: skalarna etykieta (int)
    zwraca: 2D numpy array shape (1, n_features)
    """
    # 1. Konwersja na tensor z wymiarem batch
    x = tf.convert_to_tensor(input_data.reshape((1, -1)), dtype=tf.float32)
    y = tf.convert_to_tensor([input_label], dtype=tf.int32)

    with tf.GradientTape() as tape:
        tape.watch(x)
        preds = model(x)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y, preds)

    grad = tape.gradient(loss, x)               # Obliczenie gradientu
    signed_grad = tf.sign(grad).numpy()         # Znak gradientu

    # 2. Generowanie przykładu adwersarialnego
    adv = x.numpy() + epsilon * signed_grad     # x' = x + epsilon * sign(gradient)

    # 3. Klippowanie do zakresu skalowanych danych
    adv = np.clip(adv, min_val, max_val)
    return adv



In [9]:
#Test na jednym przykładowym obrazie
idx = 0
orig = X_test[idx]
orig_label = y_test[idx]
epsilon_test = 0.8 # Siła ataku (epsilon)

pred_orig = model.predict(orig.reshape(1, -1), verbose=0)
# Używamy modelu bazowego do generowania ataku
adv_example = create_adversarial_example(model, orig, orig_label, epsilon_test, min_val, max_val)
pred_adv = model.predict(adv_example, verbose=0)

print("== Test FGSM na pojedynczym przykładzie ==")
print(f"Prawdziwa etykieta: {orig_label}")
print(f"Oryginalna predykcja: {np.argmax(pred_orig)} (konf: {np.max(pred_orig):.3f})")
print(f"Adwersarialna predykcja: {np.argmax(pred_adv)} (konf: {np.max(pred_adv):.3f})")
print(f"Czy atak się powiódł? {np.argmax(pred_orig) != np.argmax(pred_adv)}")

== Test FGSM na pojedynczym przykładzie ==
Prawdziwa etykieta: 0
Oryginalna predykcja: 0 (konf: 0.997)
Adwersarialna predykcja: 1 (konf: 0.531)
Czy atak się powiódł? True


In [10]:
#Wyszukiwanie przykładu o najniższej pewności
min_confidence = 1.0
best_idx_for_attack = -1
predictions = model.predict(X_test, verbose=0)
true_labels = y_test

# Przeszukujemy cały zbiór testowy, by znaleźć najsłabszy punkt
for i in range(len(X_test)):
    pred = predictions[i]
    confidence = np.max(pred)

    # Warunek: model musi poprawnie sklasyfikować przykład ORAZ mieć najmniejszą pewność
    if np.argmax(pred) == true_labels[i] and confidence < min_confidence:
        min_confidence = confidence
        best_idx_for_attack = i

if best_idx_for_attack != -1:
    idx = best_idx_for_attack
    print(f" Znaleziono najlepszy indeks dla ataku: {idx}")
    print(f"   Prawdziwa etykieta: {y_test[idx]}")
    print(f"   Pewność oryginalnej predykcji: {min_confidence:.4f}")
else:
    # W rzadkich przypadkach, gdy wszystkie przykłady mają idealną pewność lub kod jest zły
    idx = 0
    print(f"⚠️ Nie znaleziono idealnego słabego punktu. Używam indeksu domyślnego: {idx}")

#Przypisanie danych do testu
orig = X_test[idx]
orig_label = y_test[idx]

#Parametry ataku
epsilon_test = 0.5 # Zwiększamy epsilon, aby skompensować ewentualną trudność ataku

#Predykcje i Generowanie Ataku
pred_orig = model.predict(orig.reshape(1, -1), verbose=0)
adv_example = create_adversarial_example(model, orig, orig_label, epsilon_test, min_val, max_val)
pred_adv = model.predict(adv_example, verbose=0)

#Wyniki Ataku
print("\n== Test FGSM na Słabym Punkcie ==")
print(f"Prawdziwa etykieta: {orig_label}")
print(f"Oryginalna predykcja: {np.argmax(pred_orig)} (konf: {np.max(pred_orig):.3f})")
print(f"Adwersarialna predykcja: {np.argmax(pred_adv)} (konf: {np.max(pred_adv):.3f})")
print(f"Czy atak się powiódł? {np.argmax(pred_orig) != np.argmax(pred_adv)}")

 Znaleziono najlepszy indeks dla ataku: 9
   Prawdziwa etykieta: 1
   Pewność oryginalnej predykcji: 0.4999

== Test FGSM na Słabym Punkcie ==
Prawdziwa etykieta: 1
Oryginalna predykcja: 1 (konf: 0.500)
Adwersarialna predykcja: 2 (konf: 0.872)
Czy atak się powiódł? True


In [11]:
# Tabela z wynikami
results_data = {
    "Metryka": ["Prawdziwa Etykieta", "Oryginalna Predykcja", "Adwersarialna Predykcja", "Sukces"],
    "Wartość": [
        orig_label,
        f"{np.argmax(pred_orig)} (pewność: {np.max(pred_orig):.3f})",
        f"{np.argmax(pred_adv)} (pewność: {np.max(pred_adv):.3f})",
        "TAK" if np.argmax(pred_orig) != np.argmax(pred_adv) else "NIE"
    ]
}

df_results = pd.DataFrame(results_data)
print("\n=== Podsumowanie Ataku ===")
display(df_results)

# Porównanie cech (odwrócenie skalowania)
orig_real = scaler.inverse_transform(orig.reshape(1, -1))[0]
adv_real = scaler.inverse_transform(adv_example)[0]

# Tłumaczenie nazw cech
feature_map = {
    "sepal length (cm)": "długość działki kielicha (cm)",
    "sepal width (cm)": "szerokość działki kielicha (cm)",
    "petal length (cm)": "długość płatka (cm)",
    "petal width (cm)": "szerokość płatka (cm)"
}
polish_features = [feature_map.get(f, f) for f in data.feature_names]

feature_comparison = {
    "Cecha": polish_features,
    "Oryginał": list(orig_real),
    "Adwersarialne": list(adv_real),
    "Różnica": list(adv_real - orig_real)
}

df_features = pd.DataFrame(feature_comparison)

# Dodanie wiersza z klasyfikacją (Gatunek)
class_names = data.target_names
idx_orig = np.argmax(pred_orig)
idx_adv = np.argmax(pred_adv)

new_row = {
    "Cecha": "PREDYKCJA (Gatunek)",
    "Oryginał": f"{idx_orig} ({class_names[idx_orig]})",
    "Adwersarialne": f"{idx_adv} ({class_names[idx_adv]})",
    "Różnica": "PORAŻKA" if idx_orig == idx_adv else "SUKCES"
}

# Append new row using pd.concat to avoid FutureWarning
df_features = pd.concat([df_features, pd.DataFrame([new_row])], ignore_index=True)

print("\n=== Porównanie Cech (Wartości Rzeczywiste) ===")
display(df_features)


=== Podsumowanie Ataku ===


,Metryka,Wartość
0,Prawdziwa Etykieta,1
1,Oryginalna Predykcja,1 (pewność: 0.500)
2,Adwersarialna Predykcja,2 (pewność: 0.872)
3,Sukces,TAK



=== Porównanie Cech (Wartości Rzeczywiste) ===


,Cecha,Oryginał,Adwersarialne,Różnica
0,długość działki kielicha (cm),5.4,5.812651,0.412651
1,szerokość działki kielicha (cm),3.0,3.217205,0.217205
2,długość płatka (cm),4.5,5.379702,0.879702
3,szerokość płatka (cm),1.5,1.879846,0.379846
4,PREDYKCJA (Gatunek),1 (versicolor),2 (virginica),SUKCES


In [12]:
# === PODSUMOWANIE I TABELA PORÓWNAWCZA ===

# 1. Konfiguracja parametrów oceny
epsilon_eval = 0.5  # Siła ataku dla ewaluacji (możesz zmienić np. na 0.2 lub 0.8)
print(f"Generowanie zbioru adwersarialnego dla całego zbioru testowego (epsilon={epsilon_eval})...")

# 2. Generowanie danych adwersarialnych dla całego X_test
X_test_adv_list = []
for i in range(len(X_test)):
    # Używamy zdefiniowanej wcześniej funkcji create_adversarial_example
    # Upewnij się, że min_val i max_val są zdefiniowane (z komórki 1)
    adv_sample = create_adversarial_example(model, X_test[i], y_test[i], epsilon_eval, min_val, max_val)
    X_test_adv_list.append(adv_sample)

# Łączenie listy w jedną macierz numpy
X_test_adv = np.vstack(X_test_adv_list)

# 3. Ewaluacja modelu
# Wyniki na czystych danych
loss_clean, acc_clean = model.evaluate(X_test, y_test, verbose=0)
# Wyniki na danych adwersarialnych
loss_adv, acc_adv = model.evaluate(X_test_adv, y_test, verbose=0)

# Obliczenie liczby poprawnych próbek
correct_clean = int(acc_clean * len(X_test))
correct_adv = int(acc_adv * len(X_test))

# 4. Tworzenie tabeli wyników
comparison_data = {
    "Metryka": ["Dokładność (Accuracy)", "Strata (Loss)", "Poprawne Próbki (na 30)"],
    "Czyste Dane": [
        f"{acc_clean:.4f}",
        f"{loss_clean:.4f}",
        f"{correct_clean}/{len(X_test)}"
    ],
    f"Dane Adwersarialne (eps={epsilon_eval})": [
        f"{acc_adv:.4f}",
        f"{loss_adv:.4f}",
        f"{correct_adv}/{len(X_test)}"
    ],
    "Różnica (Wpływ ataku)": [
        f"{(acc_adv - acc_clean):.4f}",
        f"{(loss_adv - loss_clean):.4f}",
        f"{correct_adv - correct_clean}"
    ]
}

df_comparison = pd.DataFrame(comparison_data)

# Wyświetlenie tabeli
print("\n=== TABELA PORÓWNAWCZA: MODEL BAZOWY VS ATAK FGSM ===")
from IPython.display import display
display(df_comparison)

# Opcjonalnie: Obliczenie skuteczności ataku (Attack Success Rate - ile próbek zmieniło klasyfikację na błędną)
print(f"\nSkuteczność ataku (spadek accuracy): {(acc_clean - acc_adv)*100:.2f} p.p.")

Generowanie zbioru adwersarialnego dla całego zbioru testowego (epsilon=0.5)...

=== TABELA PORÓWNAWCZA: MODEL BAZOWY VS ATAK FGSM ===


,Metryka,Czyste Dane,Dane Adwersarialne (eps=0.5),Różnica (Wpływ ataku)
0,Dokładność (Accuracy),0.8667,0.5667,-0.3000
1,Strata (Loss),0.2559,0.9807,0.7249
2,Poprawne Próbki (na 30),26/30,16/30,-10



Skuteczność ataku (spadek accuracy): 30.00 p.p.
